In [1]:
# Import necessary libraries
from pyspark.sql import SparkSession
from pyspark.sql.functions import hour, sum as _sum, col

# Create SparkSession
spark = SparkSession.builder \
    .appName("Retail Sales Analysis") \
    .config("spark.driver.memory", "2g") \
    .getOrCreate()


In [2]:
# Load the dataset
data = spark.read.csv("Downloads/SupermartSales.csv", header=True, inferSchema=True)

# Show the first few rows to verify the data
data.show(5)


+--------+-------------+----------------+----------------+-----------+----------+------+-----+--------+------+----------+
|Order ID|Customer Name|        Category|    Sub Category|       City|Order Date|Region|Sales|Discount|Profit|     State|
+--------+-------------+----------------+----------------+-----------+----------+------+-----+--------+------+----------+
|     OD1|       Harish|    Oil & Masala|         Masalas|    Vellore|11-08-2017| North| 1254|    0.12|401.28|Tamil Nadu|
|     OD2|        Sudha|       Beverages|   Health Drinks|Krishnagiri|11-08-2017| South|  749|    0.18| 149.8|Tamil Nadu|
|     OD3|      Hussain|     Food Grains|    Atta & Flour| Perambalur|06-12-2017|  West| 2360|    0.21| 165.2|Tamil Nadu|
|     OD4|      Jackson|Fruits & Veggies|Fresh Vegetables| Dharmapuri|10-11-2016| South|  896|    0.25|  89.6|Tamil Nadu|
|     OD5|      Ridhesh|     Food Grains| Organic Staples|       Ooty|10-11-2016| South| 2355|    0.26|918.45|Tamil Nadu|
+--------+-------------+

In [ ]:
# Data Preprocessing

In [15]:
from pyspark.sql.functions import to_date, col, hour

# Parse the "Order Date" in the format MM/d/yyyy (without leading zeros for single digits)
data = data.withColumn("OrderDate", to_date("Order Date", "M/d/yyyy"))

# Show the processed data
data.select("Order Date", "OrderDate").show(5)


+----------+----------+
|Order Date| OrderDate|
+----------+----------+
| 8/25/2015|2015-08-25|
| 9/12/2017|2017-09-12|
| 3/16/2016|2016-03-16|
| 10/3/2016|2016-10-03|
|  9/9/2015|2015-09-09|
+----------+----------+
only showing top 5 rows



In [12]:
# Goal 1
# Identify top-performing products and their seasonal trends

In [17]:
from pyspark.sql.functions import col, sum as _sum

# Group by ProductName and calculate total sales
product_sales = data.groupBy("Category").agg(_sum("Sales").alias("TotalSales"))

# Sort by TotalSales in descending order
product_sales = product_sales.orderBy(col("TotalSales").desc())

# Show the top-performing products
product_sales.show(10)


+-----------------+----------+
|         Category|TotalSales|
+-----------------+----------+
|Eggs, Meat & Fish|   2267401|
|           Snacks|   2237546|
|      Food Grains|   2115272|
|           Bakery|   2112281|
| Fruits & Veggies|   2100727|
|        Beverages|   2085313|
|     Oil & Masala|   2038442|
+-----------------+----------+



In [9]:
#Goal 2
#Determine peak sales periods and customer purchasing behaviors

In [23]:
from pyspark.sql.functions import month, when, col, sum as _sum
from pyspark.sql import SparkSession

# Assuming you have already initialized your SparkSession
spark = SparkSession.builder.appName("SeasonalSalesAnalysis").getOrCreate()

# Convert Order Date to date type (if not already)
data = data.withColumn("OrderDate", to_date("Order Date", "M/d/yyyy"))

# Extract the month from the Order Date
data = data.withColumn("Month", month("OrderDate"))

# Map months to seasons
data = data.withColumn(
    "Season",
    when(col("Month").between(12, 2), "Winter")
    .when(col("Month").between(3, 5), "Summer")
    .when(col("Month").between(6, 9), "Monsoon")
    .otherwise("Post-Monsoon/Autumn")  # For months 10 and 11
)

# Group by season and calculate the total sales for each season
seasonal_sales = data.groupBy("Season").agg(_sum("Sales").alias("TotalSales"))

# Sort by TotalSales in descending order to find the peak season
seasonal_sales = seasonal_sales.orderBy("TotalSales", ascending=False)

# Show the peak sales periods for each season
seasonal_sales.show()

# Retrieve the season with the highest sales
peak_sales = seasonal_sales.limit(1)

# Print the peak sales season
peak_sales.show(truncate=False)


+-------------------+----------+
|             Season|TotalSales|
+-------------------+----------+
|Post-Monsoon/Autumn|   6559363|
|            Monsoon|   5258266|
|             Summer|   3139353|
+-------------------+----------+

+-------------------+----------+
|Season             |TotalSales|
+-------------------+----------+
|Post-Monsoon/Autumn|6559363   |
+-------------------+----------+



In [ ]:
# Goal 3
# Segment customers based on their purchasing behavior, such as purchase frequency, average spending, and product preferences

In [34]:
from pyspark.ml.clustering import KMeans
from pyspark.ml.feature import VectorAssembler
from pyspark.sql.functions import count, avg, sum as _sum

# Aggregate customer metrics (TotalSpend, PurchaseFrequency, AvgSpend)
customer_data = data.groupBy("Customer Name").agg(
    _sum("Sales").alias("TotalSpend"),
    count("Order ID").alias("PurchaseFrequency"),
    avg("Sales").alias("AvgSpend")
)

# Prepare data for clustering
vector_assembler = VectorAssembler(inputCols=["TotalSpend", "PurchaseFrequency", "AvgSpend"], outputCol="features")
customer_data_features = vector_assembler.transform(customer_data)

# Apply K-Means clustering
kmeans = KMeans(k=3, seed=1)  # 3 clusters
model = kmeans.fit(customer_data_features)
clusters = model.transform(customer_data_features)

# Show clusters
clusters.select("Customer Name", "TotalSpend", "PurchaseFrequency", "AvgSpend", "prediction").show(10)


+-------------+----------+-----------------+------------------+----------+
|Customer Name|TotalSpend|PurchaseFrequency|          AvgSpend|prediction|
+-------------+----------+-----------------+------------------+----------+
|       Roshan|    298463|              201|1484.8905472636816|         2|
|        Vinne|    319565|              203|1574.2118226600985|         0|
|        James|    305915|              197|1552.8680203045685|         0|
|        Yadav|    273162|              185|1476.5513513513513|         1|
|      Willams|    293597|              195| 1505.625641025641|         2|
|       Amrish|    333351|              227|1468.5066079295154|         0|
|       Sundar|    287151|              187| 1535.566844919786|         2|
|         Ravi|    305591|              200|          1527.955|         0|
|       Sheeba|    308720|              195| 1583.179487179487|         0|
|       Harish|    293839|              208|         1412.6875|         2|
+-------------+----------

In [35]:
#Goal 4
#Profit Margin Analysis
#We will calculate the profit margin for each Category based on Sales and Profit

In [40]:
# Calculate Profit Margin for each product category
profit_data = data.groupBy("Category").agg(
    _sum("Profit").alias("TotalProfit"),
    _sum("Sales").alias("TotalSales")
).withColumn("ProfitMargin", col("TotalProfit") / col("TotalSales") * 100)

# Show top profitable categories
profit_data.orderBy(col("TotalProfit").desc()).show(10)


+-----------------+-----------------+----------+------------------+
|         Category|      TotalProfit|TotalSales|      ProfitMargin|
+-----------------+-----------------+----------+------------------+
|           Snacks|568178.8500000007|   2237546|25.392946111498965|
|Eggs, Meat & Fish|567357.2199999997|   2267401|25.022359079845153|
| Fruits & Veggies|530400.3800000008|   2100727|25.248420189772435|
|      Food Grains|529162.6400000006|   2115272|25.016292940104183|
|           Bakery|528521.0600000002|   2112281|25.021342330873598|
|        Beverages|525605.7600000001|   2085313|25.205125561486458|
|     Oil & Masala|497895.2900000002|   2038442| 24.42528607632693|
+-----------------+-----------------+----------+------------------+



In [41]:
# GOAL 5: Profitability Analysis by Product 
#This will identify the most and least profitable products in terms of their profit margins.
profit_by_product = data.groupBy("Category", "Sub Category").agg(
    _sum("Profit").alias("TotalProfit"),
    _sum("Sales").alias("TotalSales")
).withColumn("ProfitMargin", col("TotalProfit") / col("TotalSales") * 100)

# Show the most profitable products
profit_by_product.orderBy(col("ProfitMargin").desc()).show(10)


+-----------------+------------------+------------------+----------+------------------+
|         Category|      Sub Category|       TotalProfit|TotalSales|      ProfitMargin|
+-----------------+------------------+------------------+----------+------------------+
|           Snacks|           Noodles|193685.81000000017|    735435|26.336224139454906|
|Eggs, Meat & Fish|              Fish| 147248.0100000001|    560548|26.268581816365433|
| Fruits & Veggies|    Organic Fruits|130862.32999999997|    503402|25.995591992085842|
|      Food Grains|   Organic Staples|144136.88999999996|    558929|25.788050002773154|
|           Bakery|     Breads & Buns|190764.98000000013|    742586| 25.68927774022135|
| Fruits & Veggies|Organic Vegetables|133596.37000000002|    520271|25.678227308460404|
|      Food Grains|              Rice|126932.35999999997|    498323|25.471904768593856|
|        Beverages|     Health Drinks| 267469.7899999999|   1051439|25.438450542542167|
|     Oil & Masala| Edible Oil &